# Home Depot Product Search Relevance IRDM Porject

## <font color='green'>Setup 1</font>: Envrionment Setup

In [1]:
from config import *

import time
start_time = time.time()

import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.mpl_style = 'default'

import re
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor,AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor

stemmer = SnowballStemmer('english')

C:\Users\xt\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


## <font color='green'>Setup 2</font>: Import Data

In [2]:
df_train = pd.read_csv(PROCESSINGTEXT_DIR + '/df_train_processed.csv', encoding="ISO-8859-1")
df_test = pd.read_csv(PROCESSINGTEXT_DIR + '/df_test_processed.csv', encoding="ISO-8859-1")
df_attr_bullet = pd.read_csv(PROCESSINGTEXT_DIR + '/df_attribute_bullets_processed.csv')
df_pro_desc = pd.read_csv(PROCESSINGTEXT_DIR + '/df_product_descriptions_processed.csv') 

num_train = df_train.shape[0]
num_test = df_test.shape[0]

df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_attr_bullet, how='left', on='product_uid')
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')

In [3]:
for var in df_all.keys():
    df_all[var]=df_all[var].fillna("")

In [4]:
df_all[:3]

,id,product_title,product_uid,relevance,search_term,brand,product_title_simpleparsed,search_term_simpleparsed,search_term_parsed,is_query_misspelled,product_title_parsed,brand_parsed,material,material_parsed,search_term_parsed_woBrand,brands_in_search_term,search_term_parsed_woBM,materials_in_search_term,product_title_parsed_woBrand,brands_in_product_title,product_title_parsed_woBM,materials_in_product_title,search_term_tokens,product_title_tokens,search_term_stemmed,search_term_stemmed_woBM,search_term_stemmed_woBrand,product_title_stemmed,product_title_stemmed_woBM,product_title_stemmed_woBrand,attribute_bullets,attribute_bullets_parsed,attribute_bullets_parsed_woBrand,brands_in_attribute_bullets,attribute_bullets_parsed_woBM,materials_in_attribute_bullets,attribute_bullets_tokens,attribute_bullets_stemmed,product_description,product_description_parsed,product_description_parsed_woBrand,brands_in_product_description,product_description_parsed_woBM,materials_in_product_description,product_description_stemmed
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3,angle bracket,Simpson Strong-Tie,simpson strong tie 12 gauge angle,angle bracket,angle bracket,1.000000,simpson strong tie 12 gauge angle,simpson,Galvanized Steel,galvanized steel,angle bracket,,angle bracket,,12 gauge angle,simpson strong tie,12 gauge angle,,"[(angle, NN), (bracket, NN)]","[(12, CD), (gauge, JJ), (angle, NN)]",angl bracket,angl bracket,angl bracket,simpson strong tie 12 gaug angl,12 gaug angl,12 gaug angl,Versatile connector for various 90Â° connectio...,versatile connector for various 90 connections...,connector for various 90 connections and home ...,versatile,connector for 90 connections and home repair p...,steel;various,"[(connector, NN), (for, IN), (90, CD), (connec...",versatil connector various 90 connect home rep...,"Not only do angles make joints stronger, they ...",not only do angles make joints stronger. they ...,not only do angles make joints stronger. they ...,simpson strong tie;versatile,not only do angles make joints stronger. they ...,steel;various;zinc,angl make joint stronger also provid consist s...
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.5,l bracket,Simpson Strong-Tie,simpson strong tie 12 gauge angle,l bracket,l bracket,1.000000,simpson strong tie 12 gauge angle,simpson,Galvanized Steel,galvanized steel,l bracket,,l bracket,,12 gauge angle,simpson strong tie,12 gauge angle,,"[(l, NN), (bracket, NN)]","[(12, CD), (gauge, NN), (angle, NN)]",l bracket,l bracket,l bracket,simpson strong tie 12 gaug angl,12 gaug angl,12 gaug angl,Versatile connector for various 90Â° connectio...,versatile connector for various 90 connections...,connector for various 90 connections and home ...,versatile,connector for 90 connections and home repair p...,steel;various,"[(connector, NN), (for, IN), (90, CD), (connec...",versatil connector various 90 connect home rep...,"Not only do angles make joints stronger, they ...",not only do angles make joints stronger. they ...,not only do angles make joints stronger. they ...,simpson strong tie;versatile,not only do angles make joints stronger. they ...,steel;various;zinc,angl make joint stronger also provid consist s...
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3,deck over,BEHR Premium Textured DeckOver,behr premium textured deckover 1 gal sc 141 tu...,deck over,deckover,0.941176,behr premium textured deckover 1-gal sc141 tug...,behr,,,deckover,,deckover,,1-gal sc141 tugboat wood and concrete coating,behr premium textured deckover,1-gal sc141 tugboat and coating,concrete;wood,"[(deckover, NN)]","[(1-gal, JJ), (sc141, NN), (tugboat, NN), (and...",deckov,deckov,deckov,behr premium textur deckov 1-gal sc141 tugboat...,1-gal sc141 tugboat coat,1-gal sc141 tugboat wood concret coat,"Revives wood and composite decks, railings, po...",revives wood and composite decks. railings. po...,revives wood and composite decks. railings. po...,behr,revives and decks. railings. porches and boat ...,acrylic;conc

In [5]:
remove_list=[str(x) for x in list(df_all.columns)]

for x in ['id','product_uid','relevance','is_query_misspelled']:
    remove_list.remove(x)

## <font color='green'>Setup 3</font>:  Feature Extraction

In [6]:
from feature_extraction_functions import *

In [7]:
import seaborn as sns

df_all['majority_relevance'] = df_all['relevance'].map(lambda x: x in [1.0, 1.33, 1.67, 2.0, 2.33, 2.67, 3.0])
def majoritize(df):
    return df[df['majority_relevance'] == 1]

sns.set_palette("husl")

Length features:

In [8]:
df_all['len_of_query'] = df_all['search_term_parsed'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title_parsed'].map(lambda x:len(x.split())).astype(np.int64)

Matching between:
- number of unique words in intersection
- number of total words in intersection
- number of letters in unique words in intersection
- ratio of common words to all words in query
- ratio of the number of letters in common words to the total number of letters in query

In [9]:
df_all['word_in_title_tuple']=df_all.apply(lambda x: \
            str_common_word(x['search_term_stemmed'],x['product_title_stemmed']),axis=1)
df_all['word_in_title_num'] = df_all['word_in_title_tuple'].map(lambda x: x[0])
df_all['word_in_title_sum'] = df_all['word_in_title_tuple'].map(lambda x: x[1])
df_all['word_in_title_let'] = df_all['word_in_title_tuple'].map(lambda x: x[2])
df_all['word_in_title_numratio'] = df_all['word_in_title_tuple'].map(lambda x: x[3])
df_all['word_in_title_letratio'] = df_all['word_in_title_tuple'].map(lambda x: x[4])
df_all['word_in_title_string'] = df_all['word_in_title_tuple'].map(lambda x: x[5])

df_all =df_all.drop(['word_in_title_tuple','word_in_title_string'],axis=1)

In [10]:
df_all['word_in_description_tuple']=df_all.apply(lambda x: \
            str_common_word(x['search_term_stemmed'],x['product_description_stemmed']),axis=1)
df_all['word_in_description_num'] = df_all['word_in_description_tuple'].map(lambda x: x[0])
df_all['word_in_description_sum'] = df_all['word_in_description_tuple'].map(lambda x: x[1])
df_all['word_in_description_let'] = df_all['word_in_description_tuple'].map(lambda x: x[2])
df_all['word_in_description_numratio'] = df_all['word_in_description_tuple'].map(lambda x: x[3])
df_all['word_in_description_letratio'] = df_all['word_in_description_tuple'].map(lambda x: x[4])
df_all['word_in_description_string'] = df_all['word_in_description_tuple'].map(lambda x: x[5])

df_all =df_all.drop(['word_in_description_tuple','word_in_description_string'],axis=1)

Measure of whether brands and materials in query match with attributes/brand in product title/etc:
* 3 if all brands/materials fully matched
* 2 if all matched at least partially
* 1 if at least one is matched but at least one is not matched
* 0 if no brand/material in query
* -1 if there is brand/material in query but no brand/material in text
* -2 if there are brands different brands/materials in query and text 

In [11]:
df_all['brands_all']=df_all['brands_in_search_term']+"\t"+df_all['brand_parsed']+"\t"+df_all['brands_in_product_title']+"\t"+\
df_all['brands_in_attribute_bullets']+"\t"+df_all['brands_in_product_description']
df_all['materials_all']=df_all['materials_in_search_term']+"\t"+df_all['material_parsed']+"\t"+df_all['materials_in_product_title']+"\t"+\
df_all['materials_in_attribute_bullets']+"\t"+df_all['materials_in_product_description']

df_all['brandinquery_brand_matching']=df_all['brands_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[1]))
df_all['brandinquery_brandintitile_matching']=df_all['brands_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[2]))
df_all['brandinquery_bullets_brand_matching']=df_all['brands_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[3]))
df_all['brandinquery_decription_brand_matching']=df_all['brands_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[4]))
                  
df_all['materialinquery_material_matching']=df_all['materials_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[1]))
df_all['materialinquery_materialintitile_matching']=df_all['materials_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[2]))
df_all['materialinquery_bullets_material_matching']=df_all['brands_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[3]))
df_all['materialinquery_decription_material_matching']=df_all['brands_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[4]))
  
df_all =df_all.drop(['brands_all','materials_all'],axis=1)

Word2Vec features:

In [12]:
df_word2vec = pd.read_csv(FEATURES_DIR + '/df_word2vec.csv') 
df_all = pd.merge(df_all, df_word2vec, how='left', on='id')

Similarity features:

In [13]:
distance_related_features =pd.read_csv(FEATURES_DIR + '/distance_related_features.csv') 
df_all = pd.merge(df_all, distance_related_features, how='left', on='id')

In [14]:
lsi = pd.read_csv(FEATURES_DIR + '/latent_semantic_indexing.csv') 
df_all = pd.merge(df_all, lsi, how='left', on='id')

Sequence Matching:

In [15]:
seq_match =pd.read_csv(FEATURES_DIR + '/seq_matching.csv') 
df_all = pd.merge(df_all, seq_match, how='left', on='id')

construct input

In [16]:
df_all =df_all.drop(['majority_relevance','edit_product_title_avg'],axis=1)

In [17]:
df_all = df_all.drop(remove_list,axis=1)

In [18]:
df_all[9:10]

,id,product_uid,relevance,is_query_misspelled,len_of_query,len_of_title,word_in_title_num,word_in_title_sum,word_in_title_let,word_in_title_numratio,word_in_title_letratio,word_in_description_num,word_in_description_sum,word_in_description_let,word_in_description_numratio,word_in_description_letratio,brandinquery_brand_matching,brandinquery_brandintitile_matching,brandinquery_bullets_brand_matching,brandinquery_decription_brand_matching,materialinquery_material_matching,materialinquery_materialintitile_matching,materialinquery_bullets_material_matching,materialinquery_decription_material_matching,word2vec_product_title_1,word2vec_product_desc_1,word2vec_bullet_1,word2vec_all_1,word2vec_product_title_2,word2vec_product_desc_2,word2vec_bullet_2,word2vec_all_2,cv_product_title_sim,cv_product_desc_sim,cv_bullet_sim,tfidf_product_title_sim,tfidf_product_desc_sim,tfidf_bullet_sim,jaccard_product_title,jaccard_product_desc,st_bow_tsvd0,st_bow_tsvd1,st_bow_tsvd2,st_bow_tsvd3,st_bow_tsvd4,st_bow_tsvd5,st_bow_tsvd6,st_bow_tsvd7,st_bow_tsvd8,st_bow_tsvd9,pt_bow_tsvd0,pt_bow_tsvd1,pt_bow_tsvd2,pt_bow_tsvd3,pt_bow_tsvd4,pt_bow_tsvd5,pt_bow_tsvd6,pt_bow_tsvd7,pt_bow_tsvd8,pt_bow_tsvd9,pd_bow_tsvd0,pd_bow_tsvd1,pd_bow_tsvd2,pd_bow_tsvd3,pd_bow_tsvd4,pd_bow_tsvd5,pd_bow_tsvd6,pd_bow_tsvd7,pd_bow_tsvd8,pd_bow_tsvd9,bl_bow_tsvd0,bl_bow_tsvd1,bl_bow_tsvd2,bl_bow_tsvd3,bl_bow_tsvd4,bl_bow_tsvd5,bl_bow_tsvd6,bl_bow_tsvd7,bl_bow_tsvd8,bl_bow_tsvd9,st_tfidf_tsvd_0,st_tfidf_tsvd_1,st_tfidf_tsvd_2,st_tfidf_tsvd_3,st_tfidf_tsvd_4,st_tfidf_tsvd_5,st_tfidf_tsvd_6,st_tfidf_tsvd_7,st_tfidf_tsvd_8,st_tfidf_tsvd_9,pt_tfidf_tsvd_0,pt_tfidf_tsvd_1,pt_tfidf_tsvd_2,pt_tfidf_tsvd_3,pt_tfidf_tsvd_4,pt_tfidf_tsvd_5,pt_tfidf_tsvd_6,pt_tfidf_tsvd_7,pt_tfidf_tsvd_8,pt_tfidf_tsvd_9,pd_tfidf_tsvd_0,pd_tfidf_tsvd_1,pd_tfidf_tsvd_2,pd_tfidf_tsvd_3,pd_tfidf_tsvd_4,pd_tfidf_tsvd_5,pd_tfidf_tsvd_6,pd_tfidf_tsvd_7,pd_tfidf_tsvd_8,pd_tfidf_tsvd_9,bl_tfidf_tsvd_0,bl_tfidf_tsvd_1,bl_tfidf_tsvd_2,bl_tfidf_tsvd_3,bl_tfidf_tsvd_4,bl_tfidf_tsvd_5,bl_tfidf_tsvd_6,bl_tfidf_tsvd_7,bl_tfidf_tsvd_8,bl_tfidf_tsvd_9,seqmatch_title_ratio,seqmatch_title_ratioscaled,seqmatch_description_ratio,seqmatch_description_ratioscaled,seqmatch_bullets_ratio,seqmatch_bullets_ratioscaled,seqmatch_desc&bullets_ratio,seqmatch_desc&bullets_ratioscaled
9,27,100009,3,1.0,2,11,1,1,3,0.5,0.5,1,1,3,0.5,0.5,0,0,0,0,3,3,0,0,0.612343,0.520279,0.574212,0.588948,0.531753,0.485249,0.542934,0.545603,0.5,0.133631,0.301511,0.432782,0.186652,0.449553,0.1,0.020833,0.000226,-0.000003,0.001087,-0.000384,-0.001225,0.000488,0.000872,0.001342,-0.001577,-0.00185,0.840624,0.480261,0.126333,-0.05363,0.058504,0.018061,-0.025332,-0.037708,-0.037676,-0.010381,2.311908,-0.623657,-1.115218,0.183772,0.579718,1.618,0.563019,0.074068,1.077308,0.197091,0.703935,-0.561067,-0.069051,-0.066946,-0.073887,0.612155,0.21395,0.210076,0.124558,0.139865,0.000509,0.000257,0.000641,0.000641,-0.00075,0.00016,0.000601,-0.000027,-0.00199,0.000317,0.125991,0.165503,-0.022922,-0.016516,0.008614,0.033024,-0.027409,0.011366,0.00371,-0.013863,0.238542,0.036778,-0.09194,0.123427,0.132426,-0.037901,-0.011963,-0.08217,-0.028969,-0.032392,0.134288,0.002263,-0.119844,0.108588,-0.065862,0.00906,0.033996,-0.01832,0.021783,0.032723,0.153846,0.989011,0.024316,1.118541,0.1,1.042857,0.019851,1.123005


In [ ]:
df_all.to_csv(FEATURES_DIR+"/df_all.csv", index=False, encoding = 'utf-8')

## <font color='green'>Setup 4</font>: Define Train/Test Sets

In [19]:
train = df_all.iloc[:num_train]
test = df_all.iloc[num_train:]
id_test = df_test['id']

y_train = train['relevance'].values
X_train = train.drop(['id','relevance'],axis=1).values
X_test = test.drop(['id','relevance'],axis=1).values

In [20]:
train.drop(['id','relevance'],axis=1)

,product_uid,is_query_misspelled,len_of_query,len_of_title,word_in_title_num,word_in_title_sum,word_in_title_let,word_in_title_numratio,word_in_title_letratio,word_in_description_num,word_in_description_sum,word_in_description_let,word_in_description_numratio,word_in_description_letratio,brandinquery_brand_matching,brandinquery_brandintitile_matching,brandinquery_bullets_brand_matching,brandinquery_decription_brand_matching,materialinquery_material_matching,materialinquery_materialintitile_matching,materialinquery_bullets_material_matching,materialinquery_decription_material_matching,word2vec_product_title_1,word2vec_product_desc_1,word2vec_bullet_1,word2vec_all_1,word2vec_product_title_2,word2vec_product_desc_2,word2vec_bullet_2,word2vec_all_2,cv_product_title_sim,cv_product_desc_sim,cv_bullet_sim,tfidf_product_title_sim,tfidf_product_desc_sim,tfidf_bullet_sim,jaccard_product_title,jaccard_product_desc,st_bow_tsvd0,st_bow_tsvd1,st_bow_tsvd2,st_bow_tsvd3,st_bow_tsvd4,st_bow_tsvd5,st_bow_tsvd6,st_bow_tsvd7,st_bow_tsvd8,st_bow_tsvd9,pt_bow_tsvd0,pt_bow_tsvd1,pt_bow_tsvd2,pt_bow_tsvd3,pt_bow_tsvd4,pt_bow_tsvd5,pt_bow_tsvd6,pt_bow_tsvd7,pt_bow_tsvd8,pt_bow_tsvd9,pd_bow_tsvd0,pd_bow_tsvd1,pd_bow_tsvd2,pd_bow_tsvd3,pd_bow_tsvd4,pd_bow_tsvd5,pd_bow_tsvd6,pd_bow_tsvd7,pd_bow_tsvd8,pd_bow_tsvd9,bl_bow_tsvd0,bl_bow_tsvd1,bl_bow_tsvd2,bl_bow_tsvd3,bl_bow_tsvd4,bl_bow_tsvd5,bl_bow_tsvd6,bl_bow_tsvd7,bl_bow_tsvd8,bl_bow_tsvd9,st_tfidf_tsvd_0,st_tfidf_tsvd_1,st_tfidf_tsvd_2,st_tfidf_tsvd_3,st_tfidf_tsvd_4,st_tfidf_tsvd_5,st_tfidf_tsvd_6,st_tfidf_tsvd_7,st_tfidf_tsvd_8,st_tfidf_tsvd_9,pt_tfidf_tsvd_0,pt_tfidf_tsvd_1,pt_tfidf_tsvd_2,pt_tfidf_tsvd_3,pt_tfidf_tsvd_4,pt_tfidf_tsvd_5,pt_tfidf_tsvd_6,pt_tfidf_tsvd_7,pt_tfidf_tsvd_8,pt_tfidf_tsvd_9,pd_tfidf_tsvd_0,pd_tfidf_tsvd_1,pd_tfidf_tsvd_2,pd_tfidf_tsvd_3,pd_tfidf_tsvd_4,pd_tfidf_tsvd_5,pd_tfidf_tsvd_6,pd_tfidf_tsvd_7,pd_tfidf_tsvd_8,pd_tfidf_tsvd_9,bl_tfidf_tsvd_0,bl_tfidf_tsvd_1,bl_tfidf_tsvd_2,bl_tfidf_tsvd_3,bl_tfidf_tsvd_4,bl_tfidf_tsvd_5,bl_tfidf_tsvd_6,bl_tfidf_tsvd_7,bl_tfidf_tsvd_8,bl_tfidf_tsvd_9,seqmatch_title_ratio,seqmatch_title_ratioscaled,seqmatch_description_ratio,seqmatch_description_ratioscaled,seqmatch_bullets_ratio,seqmatch_bullets_ratioscaled,seqmatch_desc&bullets_ratio,seqmatch_desc&bullets_ratioscaled
0,100001,1.000000,2,6,1,1,4,0.500000,0.363636,1,3,4,0.500000,0.363636,0,0,0,0,0,0,0,0,0.458784,0.553634,0.533719,0.549535,0.483853,0.575611,0.551408,0.570924,0.316228,0.196960,0.116248,0.271231,0.192254,0.111598,0.142857,0.014085,5.191693e-03,7.733106e-04,8.008886e-03,-9.417227e-04,6.788371e-03,6.536005e-03,6.752236e-03,1.024724e-02,-5.226411e-03,8.076031e-03,0.163678,-0.043543,-0.223910,0.427479,-0.380212,0.022372,-0.001904,0.333956,-0.018845,0.352031,2.892841,-0.545631,0.074810,-0.146206,0.506657,0.388687,-0.067663,-0.176000,-0.601222,-0.714585,0.980155,-0.092921,-0.336164,-0.470859,-0.184183,-0.310238,-0.185504,-0.333228,-0.115051,0.112784,6.101281e-03,4.145653e-03,2.970258e-03,-3.422055e-03,5.990477e-03,7.316428e-03,2.442720e-03,7.794200e-04,-3.323968e-04,1.513175e-02,0.068977,0.022891,0.031235,0.044933,-0.054612,-0.029514,0.097865,-0.059479,-0.059466,0.038606,0.180881,-0.019319,0.005699,0.051542,-0.083429,-0.072929,-0.129160,0.037023,-0.124322,-0.021425,0.133652,-0.034799,-0.021174,-0.039607,0.054978,-0.122307,-0.013745,-0.087655,-0.047306,-0.005254,0.186047,0.480620,0.027165,1.306169,0.000000,0.000000,0.018670,1.314665
1,100001,1.000000,2,6,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0.311565,0.453877,0.465730,0.456067,0.268701,0.451644,0.472438,0.452242,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.260202e-03,1.558596e-03,5.595447e-03,2.826872e-04,1.355302e-03,4.263715e-03,5.759169e-03,5.415061e-03,-3.193212e-03,7.819912e-03,0.163678,-0.043543,-0.223910,0.427479,-0.380212,0.022372,-0.001904,0.333956,-0.018845,0.352031,2.892841,-0.545631,0.074810,-0.146206,0.506657,0.388687,-0.067663,-0.176000,-0.601222,-0.714585,0.9801

## <font color='green'>Setup 5</font>: Construct the Model (Regression)

randomforest

In [ ]:
t_1 = time.time()

clf = RandomForestRegressor(n_estimators=200, max_depth=20, random_state=2017)
#rf = RandomForestRegressor(n_estimators=50, max_depth=15, random_state=0)
#clf = BaggingRegressor(clf, n_estimators=45, max_samples=0.1, random_state=25)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

'model training time:',round((time.time()-t_1)/60,1) ,'minutes\n'
#pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)

xgboost

In [21]:
import os  
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'  
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']  

import numpy as np  
import xgboost as xgb 

t_1 = time.time()

clf = xgb.XGBRegressor(max_depth = 6, n_estimators = 90, seed=2017)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

'model training time:',round((time.time()-t_1)/60,1) ,'minutes\n'

('model training time:', 0.4, 'minutes\n')

simple neural network

AdaBoost

Feature importance

In [ ]:
shown_num = 15

importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
columns = list(df_all.columns)

for i in ['id','relevance']:
    columns.remove(i)
columns = np.array(columns)

for f in range(shown_num):
    print("%d. feature %s (%f)" % (f + 1, columns[indices[f]], importances[indices[f]]))
    
plt.figure(figsize=(20, 10))
plt.barh(np.arange(shown_num), importances[indices[:shown_num]][::-1],color="b", align="center")
plt.yticks(range(shown_num), columns[indices[:shown_num]][::-1], fontsize=20)
plt.show()

## <font color='green'>Setup 6</font>: Evaluation

In [22]:
from sklearn.metrics import mean_squared_error, make_scorer

def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

df_score = pd.read_csv(os.getcwd() + '/data/solution.csv', encoding="ISO-8859-1")
public_score = df_score[df_score.Usage == 'Public']
private_score = df_score[df_score.Usage == 'Private']

public_pred = pd.DataFrame({"id": id_test, "relevance": y_pred})
public_pred = public_pred[df_score.Usage == 'Public']
private_pred = pd.DataFrame({"id": id_test, "relevance": y_pred})
private_pred = private_pred[df_score.Usage == 'Private']

In [23]:
'public score: ',fmean_squared_error(list(public_pred.relevance), list(public_score.relevance)), 'private score: ',fmean_squared_error(list(private_pred.relevance), list(private_score.relevance))

('public score: ', 0.45562915382061248, 'private score: ', 0.45507570807890402)